In [1]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import tarfile
from torch import nn
import shutil

/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn

import torch
import random
import numpy as np
import os
import json
import pandas as pd
import time
from humanfriendly import format_timespan


from advbench import datasets
from advbench import algorithms
from advbench import attacks
from advbench import hparams_registry
from advbench.lib import misc, meters, plotting, logging
from torch.cuda.amp import autocast
from torchsummary import summary

In [3]:
def load_weights(weight_path, dataset, device="cuda:1"):       
        algorithm = ERM(
        dataset.INPUT_SHAPE, 
        dataset.NUM_CLASSES,
        hparams,
        device).to(device)
        w_dict = torch.load(weight_path,map_location='cuda:1')
        #print(weight_dict.keys())
        #if db=="mnist":
        weight_dict = {}
        for s,v in w_dict.items():
                if "classifier" in s and "model" not in s:
                        weight_dict[s.replace('classifier', 'classifier.model')] = v
                else:
                        weight_dict[s] = v 
        algorithm.load_state_dict(weight_dict)
        return algorithm

In [22]:
if False:
    api = wandb.Api(timeout=50)
    dsets = ["CIFAR100", "MNIST", "STL10"]
    #results_dfs = {}
    entity = "hounie"
    all_runs_list = []
    for dataset in dsets:
        print("Fetching data for {}".format(dataset))
        project = f"DAug-{dataset}"
        base = f"../trained_weights/{project}"
        Path( base ).mkdir( parents=True, exist_ok=True )
        runs = api.runs(f"{entity}/{project}")
        print(f"{len(runs)} runs found")
        for run in runs:
            if run.state == "finished":
                id = run.id
                if True:
                    run = api.run(f"{entity}/{project}/{id}")
                    tmp_path = os.path.join(base, f"tmp")
                    weight_dir = os.path.join(base, run.name)
                    path = os.path.join(weight_dir, f"{run.id}.pkl")
                    Path( weight_dir ).mkdir( parents=True, exist_ok=True )
                    results = {**run.summary, **run.config, "weights": path, "id": run.id, "name":run.name}
                    try:
                        f = run.file("train-output/delta hist_ckpt.pkl").download(tmp_path, replace=True) 
                        os.rename(f.name, path)
                        all_runs_list.append(results)
                    except:
                        try:
                            run = api.run(f"{entity}/{project}/{id}")
                            f = run.file("train-output/loss_ckpt.pkl").download(tmp_path, replace=True)
                            os.rename(f.name, path)
                            all_runs_list.append(results)
                        except:
                            try:
                                run = api.run(f"{entity}/{project}/{id}")
                                f = run.file("train-output/acceptance rate_ckpt.pkl").download(tmp_path, replace=True)
                                os.rename(f.name, path)
                                all_runs_list.append(results)
                            except:
                                print(f"{run.name} {run.id} failed")
                                pass
            #break
    df = pd.DataFrame(all_runs_list)
    df.to_csv(f"./results.csv")

Fetching data for CIFAR100
62 runs found
TAUG Worst_DALE_PD_Reverse wrn-28-10 0 1at1dctp failed
Fetching data for MNIST
202 runs found
Fetching data for STL10
99 runs found
restful-leaf-110 o30zljoc failed


In [4]:
df = pd.read_csv("./results.csv")
test_keys = []
train_keys = []
for c in df.columns:
        if "loss" not in c and "acc" not in c:
            if c.startswith("test"):
                test_keys.append(c)
                #print(c)
            else:
                train_keys.append(c)
    

In [ ]:
from tqdm import tqdm
data_dir = '../advbench/data'
device = "cuda:1"
t_attacks = ["Worst_Of_K",]
for exp_id in tqdm(df["id"]):
    if True:
        api = wandb.Api(timeout=50)
        exp = df[df["id"]==exp_id]
        dataset =  exp["dataset"].values[0]
        algorithm = "ERM"
        perturbation = exp["perturbation"].values[0]
        model = exp["model"].values[0]
        if perturbation == "SE" and "rot" not in model:
            project = f"DAug-{dataset}"
            print(dataset, perturbation)
            t_hparams = exp[test_keys].to_dict(orient='index')[exp.index.values[0]]
            test_hparams = {}
            for k, v in t_hparams.items():
                test_hparams[k.replace("test_","")] = v

            hparams = exp[train_keys].to_dict(orient='index')[exp.index.values[0]]
            hparams['model'] = model
            hparams['epsilon'] = torch.tensor([hparams[f'epsilon_{i}'] for i in ("rot","tx","ty")]).to(device)
            test_hparams['epsilon'] = hparams['epsilon']
            hparams['batched'] = False
            test_hparams['batched'] = False
            hparams['worst_of_k_steps'] =  100
            test_hparams['worst_of_k_steps'] =  100
            aug = hparams["augment"]
            dataset = vars(datasets)[dataset](data_dir, augmentation = aug)
            train_ldr, val_ldr, test_ldr = datasets.to_loaders(dataset, hparams, device=device)
            kw_args = {"perturbation": perturbation}
            #algorithm = vars(algorithms)[algorithm](
             #   dataset.INPUT_SHAPE, 
              #  dataset.NUM_CLASSES,
               # hparams,
                #device,
                #**kw_args).to(device)
            algorithm = load_weights(exp["weights"].values[0], dataset, device="cuda:1")

            test_attacks = {
                "Worst_Of_100": attacks.Worst_Of_K(algorithm.classifier, test_hparams, device, perturbation="SE")}

            #train_clean_acc, train_clean_loss = misc.accuracy_loss(algorithm, val_ldr, device)
            #run = api.run(f"hounie/{project}/{exp_id}")
            wandb.init(project=project, resume=exp_id)
            #wandb.log({'train_clean_acc_nb': train_clean_acc, 'train_clean_loss_nb': train_clean_loss})
            # compute save and log adversarial accuracies on validation/test sets
            for attack_name, attack in test_attacks.items():
                test_adv_acc, test_adv_acc_mean, adv_loss, accs, loss, deltas = misc.adv_accuracy_loss_delta(algorithm, test_ldr, device, attack)
                train_adv_acc, train_adv_acc_mean, train_adv_loss, train_accs, train_loss, train_deltas = misc.adv_accuracy_loss_delta(algorithm, val_ldr, device, attack)
                print(f"Logging {attack_name}...")
                wandb.log({'test_acc_adv_'+attack_name: test_adv_acc, 'test_loss_adv_'+attack_name: adv_loss,
                'test_loss_adv_mean_'+attack_name: loss.mean()})
                wandb.log({'train_acc_adv_'+attack_name: train_adv_acc,'train_loss_adv_'+attack_name: train_adv_loss,
                'train_loss_adv_mean_'+attack_name: loss.mean()})
            wandb.finish(quiet=True)
            

CIFAR100 SE
Files already downloaded and verified
5000
model wrn-28-10
Using WRN-28-10


wandb: Currently logged in as: hounie. Use `wandb login --relogin` to force relogin


1000it [10:13,  1.63it/s]
500it [05:06,  1.63it/s]


Logging Worst_Of_100...


test_acc_adv_Worst_Of_100,▁
test_loss_adv_Worst_Of_100,▁
test_loss_adv_mean_Worst_Of_100,▁
train_acc_adv_Worst_Of_100,▁
train_loss_adv_Worst_Of_100,▁
train_loss_adv_mean_Worst_Of_100,▁
epoch,199
loss_avg,0.01097
lr,0.0008
mean_test_acc_advFo_Adam,50
mean_test_acc_advFo_PGD,70


...

Logging Worst_Of_100...


test_acc_adv_Worst_Of_100,▁
test_loss_adv_Worst_Of_100,▁
test_loss_adv_mean_Worst_Of_100,▁
train_acc_adv_Worst_Of_100,▁
train_loss_adv_Worst_Of_100,▁
train_loss_adv_mean_Worst_Of_100,▁
epoch,199
loss_avg,0.12976
lr,0.0008
mean_test_acc_advFo_Adam,20
mean_test_acc_advFo_PGD,50


CIFAR100 SE
Files already downloaded and verified
5000
model wrn-28-10
Using WRN-28-10


1000it [10:16,  1.62it/s]
500it [05:08,  1.62it/s]


Logging Worst_Of_100...
